## Alpaca Trading Bot ## 


### Import libraries ###

In [1]:
# Initial imports
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd
# Load .env environment variables
load_dotenv()

True

### Read in Alpaca API ###

In [2]:
import alpaca_trade_api as tradeapi

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_PAPER = True

In [3]:
 # Create a connection to the API 
api = REST(key_id=API_KEY,secret_key=API_SECRET,base_url="https://paper-api.alpaca.markets")

### View Positions ###

### Read in BTC Data from ALPACA API ###

In [4]:
# Fetch 1Minute historical bars of Bitcoin
btc_df = api.get_crypto_bars("BTCUSD", TimeFrame.Minute).df

# Filter data by exchange
btc_df = btc_df[btc_df.exchange == 'CBSE']

#Display btc data
btc_df.head()

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2022-12-05 06:00:00+00:00,CBSE,17344.70,17351.01,17337.11,17339.17,26.110700,435,17345.678593
2022-12-05 06:01:00+00:00,CBSE,17339.03,17347.16,17338.54,17339.73,7.180140,211,17342.143764
2022-12-05 06:02:00+00:00,CBSE,17340.85,17345.26,17338.58,17340.49,9.622658,209,17340.976900
2022-12-05 06:03:00+00:00,CBSE,17339.51,17342.41,17331.33,17332.33,14.138804,311,17335.978662
2022-12-05 06:04:00+00:00,CBSE,17332.33,17339.62,17328.08,17338.18,16.298333,266,17333.301699


### Clean data and calculate Bitcoin daily returns ### 

In [5]:
# Filter the date index and close columns
btc_signals_df =  btc_df.loc[:, ["close"]]

# Use the pct_change function to generate  returns from close prices
btc_signals_df["Actual Returns"] = btc_df["close"].pct_change()

# Drop all NaN values from the DataFrame
btc_signals_df = btc_signals_df.dropna()

# Review the DataFrame
display(btc_signals_df.head())
display(btc_signals_df.tail())


,close,Actual Returns
timestamp,,
2022-12-05 06:01:00+00:00,17339.73,0.000032
2022-12-05 06:02:00+00:00,17340.49,0.000044
2022-12-05 06:03:00+00:00,17332.33,-0.000471
2022-12-05 06:04:00+00:00,17338.18,0.000338
2022-12-05 06:05:00+00:00,17330.69,-0.000432


,close,Actual Returns
timestamp,,
2022-12-05 11:50:00+00:00,17304.66,0.000354
2022-12-05 11:51:00+00:00,17306.30,0.000095
2022-12-05 11:52:00+00:00,17310.30,0.000231
2022-12-05 11:53:00+00:00,17305.48,-0.000278
2022-12-05 11:54:00+00:00,17305.14,-0.000020


### Add SMA trading strategy ###

In [6]:
# Generate trading signals using short- and long-window and SMA_200 SMA values

# Set the short window and long window
short_window = 10
long_window = 100
sma_200 = 200

# Generate the fast and slow simple moving averages (10 and 100 days, respectively)
btc_signals_df['SMA_Fast'] = btc_signals_df['close'].rolling(window=short_window).mean()
btc_signals_df['SMA_Slow'] = btc_signals_df['close'].rolling(window=long_window).mean()
btc_signals_df['SMA_200'] = btc_signals_df['close'].rolling(window=sma_200).mean()


# Review the DataFrame
display(btc_signals_df.head())
display(btc_signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200
timestamp,,,,,
2022-12-05 06:01:00+00:00,17339.73,0.000032,NaN,NaN,NaN
2022-12-05 06:02:00+00:00,17340.49,0.000044,NaN,NaN,NaN
2022-12-05 06:03:00+00:00,17332.33,-0.000471,NaN,NaN,NaN
2022-12-05 06:04:00+00:00,17338.18,0.000338,NaN,NaN,NaN
2022-12-05 06:05:00+00:00,17330.69,-0.000432,NaN,NaN,NaN


,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200
timestamp,,,,,
2022-12-05 11:50:00+00:00,17304.66,0.000354,17297.234,17303.5896,17319.10055
2022-12-05 11:51:00+00:00,17306.30,0.000095,17298.020,17303.4792,17318.74405
2022-12-05 11:52:00+00:00,17310.30,0.000231,17299.967,17303.4956,17318.47095
2022-12-05 11:53:00+00:00,17305.48,-0.000278,17301.043,17303.4666,17318.18415
2022-12-05 11:54:00+00:00,17305.14,-0.000020,17302.127,17303.3981,17317.87610


### Initiate Signals ###

In [7]:
# Initialize the new Signal column
btc_signals_df['Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
btc_signals_df.loc[(btc_signals_df['Actual Returns'] >= 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
btc_signals_df.loc[(btc_signals_df['Actual Returns'] < 0), 'Signal'] = -1

# Review the DataFrame
display(btc_signals_df.head())
display(btc_signals_df.tail())

,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200,Signal
timestamp,,,,,,
2022-12-05 06:01:00+00:00,17339.73,0.000032,NaN,NaN,NaN,1.0
2022-12-05 06:02:00+00:00,17340.49,0.000044,NaN,NaN,NaN,1.0
2022-12-05 06:03:00+00:00,17332.33,-0.000471,NaN,NaN,NaN,-1.0
2022-12-05 06:04:00+00:00,17338.18,0.000338,NaN,NaN,NaN,1.0
2022-12-05 06:05:00+00:00,17330.69,-0.000432,NaN,NaN,NaN,-1.0


,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200,Signal
timestamp,,,,,,
2022-12-05 11:50:00+00:00,17304.66,0.000354,17297.234,17303.5896,17319.10055,1.0
2022-12-05 11:51:00+00:00,17306.30,0.000095,17298.020,17303.4792,17318.74405,1.0
2022-12-05 11:52:00+00:00,17310.30,0.000231,17299.967,17303.4956,17318.47095,1.0
2022-12-05 11:53:00+00:00,17305.48,-0.000278,17301.043,17303.4666,17318.18415,-1.0
2022-12-05 11:54:00+00:00,17305.14,-0.000020,17302.127,17303.3981,17317.87610,-1.0


### Pull Account data from Alpaca ###

In [8]:
# get our account data from alpaca 
account = api.get_account()
#Display Account Data
account

Account({   'account_blocked': False,
    'account_number': 'PA3YR0B1DXK2',
    'accrued_fees': '0',
    'balance_asof': '2022-12-02',
    'bod_dtbp': '0',
    'buying_power': '1851346.58672361232',
    'cash': '925673.29336180616',
    'created_at': '2022-11-24T09:23:13.041152Z',
    'crypto_status': 'ACTIVE',
    'crypto_tier': 0,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'effective_buying_power': '1851346.58672361232',
    'equity': '925673.29336180616',
    'id': 'fab3a39a-30c4-4f96-8698-c62bf476488f',
    'initial_margin': '0',
    'last_equity': '1018885.10899616125',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '923673.29',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '925673.29336180616',
    'position_market_value': '0',
    'regt_buying_power': '1851346.58672361232',
    'short_ma

### Determine BTC portfolio weight & buying power ###

In [9]:

account.equity
equity = account.equity
equity = float(equity)
btc_portfolio_weight = 0.30 * equity

non_marginable_buying_power = account.non_marginable_buying_power
non_marginable_buying_power= float(non_marginable_buying_power)
btc_signals_df['non_marginable_buying_power'] = non_marginable_buying_power
btc_signals_df.tail()

,close,Actual Returns,SMA_Fast,SMA_Slow,SMA_200,Signal,non_marginable_buying_power
timestamp,,,,,,,
2022-12-05 11:50:00+00:00,17304.66,0.000354,17297.234,17303.5896,17319.10055,1.0,923673.29
2022-12-05 11:51:00+00:00,17306.30,0.000095,17298.020,17303.4792,17318.74405,1.0,923673.29
2022-12-05 11:52:00+00:00,17310.30,0.000231,17299.967,17303.4956,17318.47095,1.0,923673.29
2022-12-05 11:53:00+00:00,17305.48,-0.000278,17301.043,17303.4666,17318.18415,-1.0,923673.29
2022-12-05 11:54:00+00:00,17305.14,-0.000020,17302.127,17303.3981,17317.87610,-1.0,923673.29


### Determine Quantity and $ value of Trade ###

In [10]:
#Pull last BTC Closing Price
btc_price = btc_signals_df['close'][-1]
btc_price

#Intialize quantity and value of trade
QTY_PER_TRADE = btc_portfolio_weight / btc_price
order_value = QTY_PER_TRADE * btc_price
order_value = float(order_value)
sell_qty_per_trade = 5
usd_price_per_trade = QTY_PER_TRADE * btc_price
print(f'The quantity of BTC per trade is {QTY_PER_TRADE} at a USD $ value of {usd_price_per_trade} with the current price of BTC at ${btc_price}') 

The quantity of BTC per trade is 16.04737020379736 at a USD $ value of 277701.98800854187 with the current price of BTC at $17305.14


### Submit Buy Order & display tranaction details ###

In [11]:
# Create a market order to buy 1 Bitcoin
if order_value <= non_marginable_buying_power:
    order_buy = api.submit_order('BTCUSD', qty=QTY_PER_TRADE, side='buy',
                            time_in_force='gtc')
    # get our account data from alpaca 
    account = api.get_account()
    # Get current position on Bitcoin. Yields error if we don't hold any
    api.get_position('BTCUSD')

    # Get all positions and find out if we hold any Bitcoin
    positions = api.list_positions()
    position_qty = 0
    for p in positions:
        if p.symbol == 'BTCUSD':
            position_qty = float(p.qty)
        
    order_value = QTY_PER_TRADE * btc_price
    non_marginable_buying_power = account.non_marginable_buying_power
    #Display Transaction details
    print(f'Updated Position: {position_qty} / Order Value from position: $ {order_value} / Portfolio buying power is now : ${non_marginable_buying_power}')
    print(f'Buy Order Transation details:')
    print(order_buy)
else: 
        print((' Side: Hold'))



Updated Position: 16.007251777 / Order Value from position: $ 277701.98800854187 / Portfolio buying power is now : $645934.55
Buy Order Transation details:
Order({   'asset_class': 'crypto',
    'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340',
    'canceled_at': None,
    'client_order_id': '31cad866-bd4b-4258-b984-d783468463b5',
    'created_at': '2022-12-05T11:55:23.117621675Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '5427f542-b356-4561-a502-29c746f7c123',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '16.047370203',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2022-12-05T11:55:23.116894815Z',
    'subtag': None,
    'symbol':

### Submit Sell Order ###

In [12]:
# Create a market order to sell 1 Bitcoin
order_sell = api.submit_order('BTCUSD', qty=1, side='sell',
                             time_in_force='gtc')
value_sold = sell_qty_per_trade * btc_price
#Display Transaction details
print(f'Updated Position: {position_qty} / Value sold from position: $ {value_sold} / Account Equity is now : ${equity} Transation details')
print(f'Order Sell Transation details:')
order_sell

Updated Position: 16.007251777 / Value sold from position: $ 86525.7 / Account Equity is now : $925673.2933618062 Transation details
Order Sell Transation details:


Order({   'asset_class': 'crypto',
    'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340',
    'canceled_at': None,
    'client_order_id': '7b1a5154-8396-4246-a6b0-197d477e61b0',
    'created_at': '2022-12-05T11:55:24.066559114Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'd6644754-1947-4f38-922d-a14e5e9d6ce4',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2022-12-05T11:55:24.065267634Z',
    'subtag': None,
    'symbol': 'BTC/USD',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2022-12-05T11:55:24.066619434Z'})

### Check account Status ###

In [13]:
# Create an account balance variable
# get our account data from alpaca 
account = api.get_account()
beginning_deposit = float(1000000.00)
portfolio_value = float(account.portfolio_value)

    #check to see if we are restricted
if account.trading_blocked:
    print('Account is currently restricted from trading.')

    #print current market value of account
    print('Total market value is ${account.portfolio_value}.')
    
    #calculate how much profit we have.
if portfolio_value >= beginning_deposit:

    profit = portfolio_value - beginning_deposit
 
    print(f"The profit from your portfolio is: ${profit}. Your current portfolio value is ${portfolio_value}")
        
else:
    print(f"There are no profits to withdraw, your current portfolio value is $ {portfolio_value} with a net loss of -${beginning_deposit - portfolio_value} ")

There are no profits to withdraw, your current portfolio value is $ 924906.1131688313 with a net loss of -$75093.88683116867 


### ALPACA TRADING BOT - SMA TRADING STRATEGY ###

In [ ]:
from datetime import datetime, timedelta
import math
import time

SYMBOL = 'BTCUSD'
SMA_FAST = short_window
SMA_SLOW = long_window
order_value= float(order_value)

# Create a pause function that re-runs to bot once every minute
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

# Same as the function in the random version
def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0

# Returns a series with the moving average
def get_sma(series, periods):
    return series.rolling(periods).mean()

# Checks whether we should buy (fast ma > slow ma)
def get_signal(fast, slow):
    print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
    return fast[-1] > slow[-1]

# Get up-to-date 1 minute data from Alpaca and add the moving averages
def get_bars(symbol):
    bars = api.get_crypto_bars(symbol, TimeFrame.Minute).df
    bars = bars[bars.exchange == 'CBSE']
    bars[f'sma_fast'] = get_sma(bars.close, SMA_FAST)
    bars[f'sma_slow'] = get_sma(bars.close, SMA_SLOW)
    return bars

while True:
    # GET DATA
    bars = get_bars(symbol=SYMBOL)
    # CHECK POSITIONS
    position = get_position(symbol=SYMBOL)
    order_value= float(order_value)
    non_marginable_buying_power= float(non_marginable_buying_power)
    position_value = position_qty * float(btc_price)
    should_buy = btc_signals_df.loc[(btc_signals_df['Actual Returns'] >= 0), 'Signal'] = 1
    should_sell = btc_signals_df.loc[(btc_signals_df['Actual Returns'] < 0), 'Signal'] = -1
    print(f"Position Size: {position} / Caulculating Buy/Sell/hold... / BTC Price: ${btc_price}")
    if order_value >= non_marginable_buying_power and should_buy == True:
        # WE BUY ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='buy',
                        time_in_force='gtc', stop_loss={'stop_price': btc_price * 0.98,})
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif order_value >= position_value and should_sell == True:
        # WE SELL ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='sell',
                        time_in_force='gtc')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {QTY_PER_TRADE}')
        
    else:
        print((f'Symbol: {SYMBOL} / Side: Hold'))
        
        

    time.sleep(get_pause())
    print("*"*20)

Position Size: 15.007251777 / Caulculating Buy/Sell/hold... / BTC Price: $17305.14
Symbol: BTCUSD / Side: Hold
Sleep for 35


#### Liquidate position ###

In [ ]:
# Get current position on Bitcoin. Yields error if we don't hold any
api.get_position('BTCUSD')

# Get all positions and find out if we hold any Bitcoin
positions = api.list_positions()
position_qty = 0
for p in positions:
    if p.symbol == 'BTCUSD':
        position_qty = float(p.qty)

# Create a market order to sell 1 Bitcoin
value_liquidated = position_qty * btc_price
liquidate_position = api.submit_order('BTCUSD', qty=position_qty, side='sell',
                             time_in_force='gtc')


#Display Transaction details
print(f'Updated Position: {position_qty} / Value Liqudated from position: $ {value_liquidated} / Account Equity is now : ${equity} Transation details')
print(f'Transation details:')
liquidate_position